In [22]:
import duckdb
import pandas as pd
from pathlib import Path

pd.set_option("display.max_colwidth", None)

BASE_DIR = Path.home() / "repos" / "Thesis"  # adjust if needed
PROJECT_DB_PATH = BASE_DIR / "data" / "duckdb" / "subs_project.duckdb"
SOURCE_DB_PATH  = BASE_DIR / "data" / "duckdb" / "subs.duckdb"

con = duckdb.connect(PROJECT_DB_PATH.as_posix(), read_only=True)

# Attach src if your view references it
dbl = con.execute("PRAGMA database_list").df()
if not (dbl["name"] == "src").any():
    con.execute(f"ATTACH '{SOURCE_DB_PATH.as_posix()}' AS src")

# IMPORTANT: make DuckDB less memory-hungry for bookkeeping
con.execute("SET preserve_insertion_order=false")
con.execute("SET threads=1")  # start low; increase later if stable

# If you can afford it, give DuckDB more headroom (optional)
con.execute("SET memory_limit='6GB'")

# datasets = ["PtBrVId", "FRMT", "Gold"]  # <- adjust names to match exactly what's in your view
# dfs = []
# for d in datasets:
#     df_d = con.execute("SELECT * FROM test_data WHERE dataset = ? LIMIT 15", [d]).df()
#     print(d, "rows:", len(df_d))
#     dfs.append(df_d)

# df = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()
# df


In [23]:
import json, re
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

THINK_RE = re.compile(r"<think>.*?</think>\s*", flags=re.DOTALL)

def clean_text(s: str) -> str:
    s = (s or "").strip()
    s = THINK_RE.sub("", s).strip()
    s = re.sub(r"\s+", " ", s)
    return s

def find_eval_results_dir(start: Path | None = None, max_up: int = 6) -> Path:
    """
    Walk upward looking for an eval_results/ directory.
    """
    p = (start or Path.cwd()).resolve()
    for _ in range(max_up + 1):
        cand = p / "eval_results"
        if cand.exists() and cand.is_dir():
            return cand
        p = p.parent
    raise FileNotFoundError("Could not find eval_results/ by walking up from current directory.")

def latest_file(eval_dir: Path, pattern: str) -> Path:
    files = sorted(eval_dir.glob(pattern), key=lambda x: x.stat().st_mtime, reverse=True)
    if not files:
        raise FileNotFoundError(f"No files matching {pattern} in {eval_dir}")
    return files[0]

eval_dir = find_eval_results_dir()
translations_file = latest_file(eval_dir, "*_translations.jsonl")

print("Using eval_results:", eval_dir)
print("Using translations file:", translations_file)


Using eval_results: /home/laiarodrigo/repos/Thesis/eval_results
Using translations file: /home/laiarodrigo/repos/Thesis/eval_results/20251229_203314_translations.jsonl


In [24]:
def load_jsonl(path: Path) -> pd.DataFrame:
    rows = []
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                rows.append(json.loads(line))
    return pd.DataFrame(rows)

df = load_jsonl(translations_file)
print("Rows:", len(df))
df.head(3)


Rows: 5222


,task,dataset,bucket,direction,source,ref,hyp
0,translate_br2pt,FRMT,entity,br2pt,(A família de Rousseff não tinha conhecimento sobre a participação dela em atividades clandestinas).,(A família de Rousseff não tinha conhecimento da participação dela em atividades ilícitas).,A família de Rousseff não tinha conhecimento sobre a participação dela em atividades clandestinas.
1,translate_pt2br,FRMT,entity,pt2br,(A família de Rousseff não tinha conhecimento da participação dela em atividades ilícitas).,(A família de Rousseff não tinha conhecimento sobre a participação dela em atividades clandestinas).,(A família de Rousseff não tinha conhecimento da participação dela em atividades ilícitas).
2,translate_br2pt,FRMT,entity,br2pt,550 a.C.,550 a.C.,550 a.C.


In [25]:
# Clean fields for fair comparisons
df["source_clean"] = df["source"].map(clean_text)
df["ref_clean"]    = df["ref"].map(clean_text)
df["hyp_clean"]    = df["hyp"].map(clean_text)

# Comparisons
df["src_eq_ref"]   = df["source_clean"] == df["ref_clean"]
df["hyp_eq_src"]   = df["hyp_clean"]    == df["source_clean"]
df["hyp_eq_ref"]   = df["hyp_clean"]    == df["ref_clean"]
df["need_change"]  = ~df["src_eq_ref"]

total = len(df)
need = int(df["need_change"].sum())

summary_counts = {
    "total_examples": total,
    "source_eq_ref": int(df["src_eq_ref"].sum()),
    "hyp_eq_source": int(df["hyp_eq_src"].sum()),
    "hyp_eq_ref": int(df["hyp_eq_ref"].sum()),
    "need_change": need,
    "need_change_but_copied": int((df["need_change"] & df["hyp_eq_src"]).sum()),
    "need_change_and_matched_ref": int((df["need_change"] & df["hyp_eq_ref"]).sum()),
}
summary_counts = pd.DataFrame([summary_counts])

summary_pct = summary_counts.copy()
for c in ["source_eq_ref", "hyp_eq_source", "hyp_eq_ref", "need_change"]:
    summary_pct[c] = (summary_counts[c] / total * 100).round(2)

need_den = max(need, 1)
summary_pct["need_change_but_copied"] = (summary_counts["need_change_but_copied"] / need_den * 100).round(2)
summary_pct["need_change_and_matched_ref"] = (summary_counts["need_change_and_matched_ref"] / need_den * 100).round(2)

print("Counts:")
display(summary_counts)
print("Percentages:")
display(summary_pct)


Counts:


,total_examples,source_eq_ref,hyp_eq_source,hyp_eq_ref,need_change,need_change_but_copied,need_change_and_matched_ref
0,5222,38,4379,36,5184,4344,1


Percentages:


,total_examples,source_eq_ref,hyp_eq_source,hyp_eq_ref,need_change,need_change_but_copied,need_change_and_matched_ref
0,5222,0.73,83.86,0.69,99.27,83.8,0.02


In [26]:
def cond_rate(group: pd.DataFrame, mask_col: str, cond_col: str) -> float:
    # rate of (cond_col True) among rows where (mask_col True)
    den = int(group[mask_col].sum())
    if den == 0:
        return 0.0
    num = int((group[mask_col] & group[cond_col]).sum())
    return num / den

by_dir = df.groupby("direction").apply(lambda g: pd.Series({
    "n": len(g),
    "hyp_eq_src_rate": g["hyp_eq_src"].mean(),
    "hyp_eq_ref_rate": g["hyp_eq_ref"].mean(),
    "need_change_rate": g["need_change"].mean(),
    "need_change_but_copied_rate": cond_rate(g, "need_change", "hyp_eq_src"),
    "need_change_and_matched_ref_rate": cond_rate(g, "need_change", "hyp_eq_ref"),
})).reset_index()

by_bucket = df.groupby("bucket").apply(lambda g: pd.Series({
    "n": len(g),
    "hyp_eq_src_rate": g["hyp_eq_src"].mean(),
    "hyp_eq_ref_rate": g["hyp_eq_ref"].mean(),
    "need_change_rate": g["need_change"].mean(),
    "need_change_but_copied_rate": cond_rate(g, "need_change", "hyp_eq_src"),
    "need_change_and_matched_ref_rate": cond_rate(g, "need_change", "hyp_eq_ref"),
})).reset_index().sort_values("n", ascending=False)

display(by_dir)
display(by_bucket.head(20))


/tmp/ipykernel_8716/77732613.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  by_dir = df.groupby("direction").apply(lambda g: pd.Series({
/tmp/ipykernel_8716/77732613.py:18: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  by_bucket = df.groupby("bucket").apply(lambda g: pd.Series({


,direction,n,hyp_eq_src_rate,hyp_eq_ref_rate,need_change_rate,need_change_but_copied_rate,need_change_and_matched_ref_rate
0,br2pt,2611.0,0.795481,0.007277,0.992723,0.794367,0.000386
1,pt2br,2611.0,0.881655,0.006511,0.992723,0.881559,0.000000


,bucket,n,hyp_eq_src_rate,hyp_eq_ref_rate,need_change_rate,need_change_but_copied_rate,need_change_and_matched_ref_rate
0,entity,1968.0,0.867886,0.010671,0.987805,0.867798,0.00000
1,lexical,1750.0,0.838286,0.001143,0.998857,0.838101,0.00000
2,random,1504.0,0.800532,0.008644,0.992021,0.798928,0.00067


In [27]:
fails = df[df["need_change"] & df["hyp_eq_src"]][
    ["dataset", "bucket", "direction", "source_clean", "ref_clean", "hyp_clean"]
]

success = df[df["need_change"] & df["hyp_eq_ref"]][
    ["dataset", "bucket", "direction", "source_clean", "ref_clean", "hyp_clean"]
].head(30)

# print("Need-change but copied (first 30):")
# display(fails)

print("Need-change and matched ref (first 30):")
display(success)



Need-change and matched ref (first 30):


,dataset,bucket,direction,source_clean,ref_clean,hyp_clean
4886,FRMT,random,br2pt,O proteus é um símbolo do patrimônio natural esloveno.,O proteus é um símbolo do património natural esloveno.,O proteus é um símbolo do património natural esloveno.


In [28]:
# changed but didn't match reference (on cases where ref differs from source)
changed_not_ref = df[
    df["need_change"] & (~df["hyp_eq_src"]) & (~df["hyp_eq_ref"])
].copy()

print("Count changed-but-not-ref:", len(changed_not_ref), "out of", len(df))

# Optional: rank by how different hyp is from source (rough proxy via length of diff)
def diff_score(row):
    # simple heuristic: abs length difference + token mismatch count
    s = row["source_clean"].split()
    h = row["hyp_clean"].split()
    # token mismatches up to min length
    mism = sum(1 for a, b in zip(s, h) if a != b)
    return abs(len(s) - len(h)) + mism

changed_not_ref["diff_score"] = changed_not_ref.apply(diff_score, axis=1)

# Show "most changed" first
top = changed_not_ref.sort_values("diff_score", ascending=False).head(20)

cols = ["dataset", "bucket", "direction", "diff_score", "source_clean", "ref_clean", "hyp_clean"]
display(top[cols])

# Print a few nicely
for i, r in enumerate(top.head(8).to_dict("records"), 1):
    print(f"\n[{i}] dataset={r['dataset']} bucket={r['bucket']} dir={r['direction']} diff_score={r['diff_score']}")
    print("SRC:", r["source_clean"])
    print("REF:", r["ref_clean"])
    print("HYP:", r["hyp_clean"])



Count changed-but-not-ref: 839 out of 5222


,dataset,bucket,direction,diff_score,source_clean,ref_clean,hyp_clean
3875,FRMT,random,pt2br,88,"Entre a morte real de Abel, no seguimento de um cancro, à glacial ultima morada de Amelia… houve um verdadeiro sopro de morte neste episódio"". Complex incluiu a aparição da atriz convidada Kim Stolz na sua lista das ""25 piores aparições de atores convidados na história da TV"". BuzFeed incluiu o episódio 45 na sua lista dos melhores episódios de Veronica Mars, criticando o enredo de Clarence Wiedman, embora escrevendo que ""podemos ver alguns gracejos de qualidade entre Logan e Cliff, duas das personagens mais divertidas da série"".","Entre a morte por câncer de Abel fora das telas e o falecimento em uma banheira de gelo de Amelia, houve um verdadeiro sopro de morte nesse episódio”. Complex qualificou a participação especial de Kim Stolz em sua lista das “25 piores participações especiais da história da TV”. O BuzzFeed classificou o episódio 45 em sua lista dos melhores episódios de Veronica Mars, criticando o enredo de Clarence Wiedman enquanto escrevia que “vemos algumas brincadeiras entre Logan e Cliff, dois dos personagens mais engraçados da série”.","Complex incluiu a aparição da atriz convidada Kim Stolz na sua lista das ""25 piores aparições de atores convidados na história da TV"". BuzFeed incluiu o episódio 45 na sua lista dos melhores episódios de Veronica Mars, criticando o enredo de Clarence Wiedman, embora escrevendo que ""podemos ver alguns gracejos de qualidade entre Logan e Cliff, duas das personagens mais divertidas da série""."
3874,FRMT,random,br2pt,86,"Entre a morte por câncer de Abel fora das telas e o falecimento em uma banheira de gelo de Amelia, houve um verdadeiro sopro de morte nesse episódio”. Complex qualificou a participação especial de Kim Stolz em sua lista das “25 piores participações especiais da história da TV”. O BuzzFeed classificou o episódio 45 em sua lista dos melhores episódios de Veronica Mars, criticando o enredo de Clarence Wiedman enquanto escrevia que “vemos algumas brincadeiras entre Logan e Cliff, dois dos personagens mais engraçados da série”.","Entre a morte real de Abel, no seguimento de um cancro, à glacial ultima morada de Amelia… houve um verdadeiro sopro de morte neste episódio"". Complex incluiu a aparição da atriz convidada Kim Stolz na sua lista das ""25 piores aparições de atores convidados na história da TV"". BuzFeed incluiu o episódio 45 na sua lista dos melhores episódios de Veronica Mars, criticando o enredo de Clarence Wiedman, embora escrevendo que ""podemos ver alguns gracejos de qualidade entre Logan e Cliff, duas das personagens mais divertidas da série"".","Em relação à morte de Abel fora das telas e ao falecimento em uma banheira de gelo de Amelia, houve um verdadeiro sopro de morte nesse episódio. Complexa qualificou a participação especial de Kim Stolz em sua lista das ""25 piores participações especiais da história da TV"". O BuzzFeed classificou o episódio 45 em sua lista dos melhores episódios de Veronica Mars, criticando o enredo de Clarence Wiedman enquanto escrevia que ""vemos algumas brincadeiras entre Logan e"
3184,FRMT,lexical,br2pt,83,"Steven Mosher, presidente do Instituto de Pesquisa da População em Washington, D.C. afirmou: ""Vinte e cinco milhões de homens na China atualmente não conseguem encontrar noivas devido a uma falta de mulheres[...] homens jovens emigram para o exterior para encontrar noivas."" O desequilíbrio de gênero nessas regiões também é culpado por incentivar o crescimento do tráfico sexual; o relatório da ONU de 2005 diz que até 800.000 pessoas são traficadas entre fronteiras a cada ano, e aproximadamente 80% delas são mulheres e garotas.","Steven Mosher, Presidente do Population Research Institute em Washington, D.C., afirmou: ""Atualmente, vinte e cinco milhões de homens na China não conseguem encontrar noiva porque há falta de mulheres (...) os jovens têm de ir para o estrangeiro para encontrar noivas"". O desequilíbrio de gé


[1] dataset=FRMT bucket=random dir=pt2br diff_score=88
SRC: Entre a morte real de Abel, no seguimento de um cancro, à glacial ultima morada de Amelia… houve um verdadeiro sopro de morte neste episódio". Complex incluiu a aparição da atriz convidada Kim Stolz na sua lista das "25 piores aparições de atores convidados na história da TV". BuzFeed incluiu o episódio 45 na sua lista dos melhores episódios de Veronica Mars, criticando o enredo de Clarence Wiedman, embora escrevendo que "podemos ver alguns gracejos de qualidade entre Logan e Cliff, duas das personagens mais divertidas da série".
REF: Entre a morte por câncer de Abel fora das telas e o falecimento em uma banheira de gelo de Amelia, houve um verdadeiro sopro de morte nesse episódio”. Complex qualificou a participação especial de Kim Stolz em sua lista das “25 piores participações especiais da história da TV”. O BuzzFeed classificou o episódio 45 em sua lista dos melhores episódios de Veronica Mars, criticando o enredo de Clare